<a href="https://colab.research.google.com/github/OAlimana/GeekBrains/blob/main/lesson_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import pandas as pd

In [55]:
user_table = pd.read_csv('user_table.csv')

# Просматриваем первые 5 строк датасета
user_table.head()

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


In [56]:
home_page = pd.read_csv('home_page_table.csv')
home_page.head()

,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page
3,290784,home_page
4,639104,home_page


In [57]:
search_page = pd.read_csv('search_page_table.csv')
search_page.head()

,user_id,page
0,15866,search_page
1,347058,search_page
2,577020,search_page
3,780347,search_page
4,383739,search_page


In [58]:
payment_page = pd.read_csv('payment_page_table.csv')
payment_page.head()

,user_id,page
0,253019,payment_page
1,310478,payment_page
2,304081,payment_page
3,901286,payment_page
4,195052,payment_page


In [59]:
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')
payment_confirmation.head()

,user_id,page
0,123100,payment_confirmation_page
1,704999,payment_confirmation_page
2,407188,payment_confirmation_page
3,538348,payment_confirmation_page
4,841681,payment_confirmation_page


In [60]:
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})

In [61]:
from functools import reduce

data_frames = [user_table, home_page, search_page, payment_page, payment_confirmation]

df = reduce(lambda  left,right: pd.merge(left,right,on='user_id', how='left'), data_frames)

df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [62]:
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)

df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [63]:
# Агрегируем данные: подсчитываем количество пользователей по половому признаку
df_merged['sex'].value_counts()

Male      45325
Female    45075
Name: sex, dtype: int64

In [64]:
# Воронка продаж по половому признаку пользователей
dfg = df.groupby('sex')[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().unstack('sex').unstack('sex').reset_index()

dfg.columns.name = None
dfg = dfg.rename(columns={'index': 'Page'})
print("Воронка продаж по половому признаку \n", dfg.head())


Воронка продаж по половому признаку 
                    Page  Female   Male
0             home_page   45075  45325
1           search_page   22676  22524
2          payment_page    3100   2930
3  payment_confirmation     241    211


In [65]:
# Добавляем к таблице месяц и день недели

df['month'] = pd.to_datetime(df['date']).dt.month
df['dayofweek'] = pd.to_datetime(df['date']).dt.day_name()
print("Таблица с двумя новыми колонками \n", df.head())

Таблица с двумя новыми колонками 
    user_id        date   device  ... payment_confirmation  month  dayofweek
0   450007  2015-02-28  Desktop  ...                    0      2   Saturday
1   756838  2015-01-13  Desktop  ...                    0      1    Tuesday
2   568983  2015-04-09  Desktop  ...                    0      4   Thursday
3   190794  2015-02-18  Desktop  ...                    0      2  Wednesday
4   537909  2015-01-15  Desktop  ...                    0      1   Thursday

[5 rows x 10 columns]


In [66]:
#Определяем самые топовые по продажам месяц и день недели
print(df[['month', 'payment_confirmation']].groupby('month').sum())
print(df[['dayofweek', 'payment_confirmation']].groupby('dayofweek').sum().sort_values(by=['payment_confirmation'], ascending=False))

       payment_confirmation
month                      
1                       189
2                       173
3                        44
4                        46
           payment_confirmation
dayofweek                      
Monday                       77
Saturday                     70
Tuesday                      69
Thursday                     66
Sunday                       62
Friday                       54
Wednesday                    54


In [67]:
#Определим в какой день недели лучше покупают мужчины, а в какой женщины
print("Талица покупок по половому признаку и дням недели\n", df.pivot_table(index=['sex', 'dayofweek'], values='payment_confirmation').unstack().unstack().unstack())

Талица покупок по половому признаку и дням недели
 sex                               Female      Male
                     dayofweek                    
payment_confirmation Friday     0.004109  0.004360
                     Monday     0.006719  0.005187
                     Saturday   0.005675  0.005266
                     Sunday     0.005380  0.004208
                     Thursday   0.004608  0.005211
                     Tuesday    0.006240  0.004613
                     Wednesday  0.004718  0.003708
